### Azure Sentiment Analysis on Wine Description

In [1]:
import csv

In [2]:
f = open('data/NLP/winemag-data_first150k.csv', encoding="utf8" )
dataset = csv.reader(f)
data_read = [row for row in dataset]

In [3]:
print(data_read[:5])

[['', 'country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'region_2', 'variety', 'winery'], ['0', 'US', 'This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.', "Martha's Vineyard", '96', '235.0', 'California', 'Napa Valley', 'Napa', 'Cabernet Sauvignon', 'Heitz'], ['1', 'Spain', 'Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.', 'Carodorum Selección Especial Reserva', '96', '110.0', 'Norther

In [4]:
index = 0
documents = {"documents":[]}
for each in data_read[1:5]:
    dataEntry = {'id': index, 'language':"en", 'text': each[2]}
    documents["documents"].append(dataEntry)
    index += 1


In [6]:
import boto3
import pandas as pd
import requests
import os
import seaborn as sns
import matplotlib.pyplot as plt

In [7]:
text_analytics_base_url = "https://westcentralus.api.cognitive.microsoft.com/text/analytics/v2.0/"

In [8]:
documents['documents'][:5]

[{'id': 0,
  'language': 'en',
  'text': 'This tremendous 100% varietal wine hails from Oakville and was aged over three years in oak. Juicy red-cherry fruit and a compelling hint of caramel greet the palate, framed by elegant, fine tannins and a subtle minty tone in the background. Balanced and rewarding from start to finish, it has years ahead of it to develop further nuance. Enjoy 2022–2030.'},
 {'id': 1,
  'language': 'en',
  'text': 'Ripe aromas of fig, blackberry and cassis are softened and sweetened by a slathering of oaky chocolate and vanilla. This is full, layered, intense and cushioned on the palate, with rich flavors of chocolaty black fruits and baking spices. A toasty, everlasting finish is heady but ideally balanced. Drink through 2023.'},
 {'id': 2,
  'language': 'en',
  'text': 'Mac Watson honors the memory of a wine once made by his mother in this tremendously delicious, balanced and complex botrytised white. Dark gold in color, it layers toasted hazelnut, pear compot

In [9]:
from pprint import pprint

In [11]:
file = open('data/rootkey.csv')
keys = csv.reader(file)
myKeys = []
for each in keys:
    for string in each:
        myKeys.append(string.split('=', 1)[1])

In [12]:
subscription_key = myKeys[2]
assert subscription_key

In [13]:
sentiment_api_url = text_analytics_base_url + "sentiment"
headers   = {"Ocp-Apim-Subscription-Key": subscription_key}
response  = requests.post(sentiment_api_url, headers=headers, json=documents)
languages = response.json()
pprint(languages)

{'documents': [{'id': '0', 'score': 0.9779012203216553},
               {'id': '1', 'score': 0.958288848400116},
               {'id': '2', 'score': 0.9469660520553589},
               {'id': '3', 'score': 0.9231853485107422}],
 'errors': []}


In [14]:
df = pd.DataFrame(languages['documents'])

In [15]:
df.head()
df.describe()

,score
count,4.000000
mean,0.951585
std,0.022843
min,0.923185
25%,0.941021
50%,0.952627
75%,0.963192
max,0.977901


In [16]:
fig, ax = plt.subplots()
sns.kdeplot(df['score'], shade=True, ax=ax)
fig.set_size_inches(11.7, 8.27)
ax.set_title("Density Plot of Sentiment Analysis for Wine Reviews via Azure NLP ")

Text(0.5,1,'Density Plot of Sentiment Analysis for Wine Reviews via Azure NLP ')

### AWS Sentiment Analysis on Wine Description

In [17]:
import json

In [18]:
comprehend = boto3.client(service_name='comprehend', region_name='us-west-2', aws_access_key_id = myKeys[0], aws_secret_access_key = myKeys[1])

In [19]:
index = 0
for each in documents['documents']:
    print('Calling ', index ,' DetectSentiment')
    print(json.dumps(comprehend.detect_sentiment(Text=each['text'], LanguageCode='en'), sort_keys=True, indent=4))
    print('End of ', index ,' DetectSentiment\n')
    index += 1

Calling  0  DetectSentiment
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "165",
            "content-type": "application/x-amz-json-1.1",
            "date": "Thu, 08 Mar 2018 09:27:39 GMT",
            "x-amzn-requestid": "f209ed3e-22b2-11e8-90b5-8b3b912ae691"
        },
        "HTTPStatusCode": 200,
        "RequestId": "f209ed3e-22b2-11e8-90b5-8b3b912ae691",
        "RetryAttempts": 0
    },
    "Sentiment": "POSITIVE",
    "SentimentScore": {
        "Mixed": 0.0021000285632908344,
        "Negative": 0.0009093565749935806,
        "Neutral": 0.13025645911693573,
        "Positive": 0.8667342066764832
    }
}
End of  0  DetectSentiment

Calling  1  DetectSentiment
{
    "ResponseMetadata": {
        "HTTPHeaders": {
            "connection": "keep-alive",
            "content-length": "164",
            "content-type": "application/x-amz-json-1.1",
            "date": "Thu, 08 Mar 2018 09:27:39 GMT",
   